In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Statistics

This notebook provides examples for extracting useful statistics from processed hdf5 data.

In [ ]:
from e11 import H5Data
from e11.stats import statistics
from e11.tools import add_column_index
from e11.process import vrange

In [ ]:
# read file
import os 
fil = os.path.join(os.getcwd(), 'example_data', 'array_data.h5')
h5 = H5Data(fil)

In [ ]:
h5.log

## Vrange

Here, we are applying the vrange function to measure the vertical range of array data.

In [ ]:
rng = h5.apply(vrange, 'OSC_0', h5.squids)
rng.head()

In [ ]:
rng.describe()

##  Statistics

The `statistics()` function can be used to find the average vrange value per squid.

In [ ]:
av = statistics(rng, mode='full')
av.head()

In [ ]:
# map measurements to var values
df = add_column_index(h5.var, 'VAR').join(av)
df.head()

In [ ]:
# plot
fig, ax = plt.subplots()

xvals = df[('VAR', 'PAUSE')]
yvals = df[('vrange', 'mean')]
yerr =  df[('vrange', 'err')]
ax.errorbar(xvals, yvals, yerr=yerr, marker='o')

#format
ax.set_xlabel('Pause (ms)')
ax.set_ylabel('signal (arb. units)')

#output
plt.show()

## Loops

It's not unusual for different squids to be run using the same experimental conditions (VARS), e.g., if looping an experiment script.

The plot above indicates that there were 2 loops recorded using the same values for VAR:PAUSE.  These can simply be grouped together to find the overall statistics for unique VAR values.

In [ ]:
# map measurements to var values
df2 = h5.var.join(rng)
df2.head()

In [ ]:
# then evaluate the statistics when grouped by var values
st = statistics(df2, groupby=h5.var.columns, mode='full')
st.head()

In [ ]:
# plot
fig, ax = plt.subplots()

xvals = st.index.get_level_values('PAUSE')
yvals = st[('vrange', 'mean')]
yerr =  st[('vrange', 'err')]
ax.errorbar(xvals, yvals, yerr=yerr, marker='o')

#format
ax.set_xlabel('pause (ms)')
ax.set_ylabel('signal (arb. units)')

#output
plt.show()